In [1]:
# 计算图
#   将代码分解成操作子
#   将计算表示成一个无环图

In [2]:
# mxnet和tensorflow可以显示构造计算图
from mxnet import sym
a = sym.var()
b = sym.var()
c = 2*a+b

OSError: dlopen(/anaconda3/envs/pytorch/lib/python3.6/site-packages/mxnet/libmxnet.dylib, 6): Symbol not found: ____chkstk_darwin
  Referenced from: /anaconda3/envs/pytorch/lib/python3.6/site-packages/mxnet/libmxnet.dylib
  Expected in: /usr/lib/libSystem.B.dylib
 in /anaconda3/envs/pytorch/lib/python3.6/site-packages/mxnet/libmxnet.dylib

In [3]:
#pytorch和mxnet可以隐式构造
from mxnet import autograd, nd
with autograd.record():#告诉系统记录构造路径
    a = nd.ones((2,1))
    b = nd.ones((2,1))
    c = 2*a+b

OSError: dlopen(/anaconda3/envs/pytorch/lib/python3.6/site-packages/mxnet/libmxnet.dylib, 6): Symbol not found: ____chkstk_darwin
  Referenced from: /anaconda3/envs/pytorch/lib/python3.6/site-packages/mxnet/libmxnet.dylib
  Expected in: /usr/lib/libSystem.B.dylib
 in /anaconda3/envs/pytorch/lib/python3.6/site-packages/mxnet/libmxnet.dylib

In [34]:
#自动求导的两种模式
  #正向累积：从变量一步步计算最后的函数的导数
  #反向累积：和正向累计相反
#复杂度
#反向计算复杂度：O(n),n是操作子个数（层数）
 #通常正向和反向的代价类似
#反向内存复杂度：O(n),需要存储正向的所有中间结果
#跟正向累积对比：
  #1.O(n)计算复杂度用来计算一个变量的梯度（正向累积时间复杂度高）
  #2.O(1)内存复杂度

In [85]:
#假设想对函数y=2X^T*X关于列向量X求导
import torch
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [86]:
#在计算y关于x的梯度之前，需要一个地方来存储梯度
x.requires_grad_(True) #等价于x=torch.arange(4.0,requires_grad=True)
x.grad

In [87]:
#计算y
#PyTorch提供的autograd包能够根据输⼊和前向传播过程⾃动构建计算图，并执⾏反向传播。
#Tensor 是这个autograd的核⼼类，如果将其属性 .requires_grad 设置为 True 
#它将开始追踪(track)在其上的所有操作（这样就可以利⽤链式法则进⾏梯度传播了）。
#完成计算后，可以调⽤ .backward() 来完成所有梯度计算。此 Tensor 的梯度将累积到 .grad 属性中
y = 2*torch.dot(x,x)
y

tensor(28., grad_fn=<MulBackward0>)

In [88]:
#通过调用反向传播函数来自动计算y关于x每个分量的梯度
y.backward()
x.grad  #计算得到y对于x每个分量的梯度

tensor([ 0.,  4.,  8., 12.])

In [89]:
x.grad == 4*x

tensor([True, True, True, True])

In [90]:
#grad在反向传播过程中是累加的(accumulated)，这意味着每⼀次运⾏反向传播，梯度都会累加之前的梯度，所以⼀般在反向传播之前需把梯度清零。
x.grad.zero_()

tensor([0., 0., 0., 0.])

In [91]:
y = x.sum()
y

tensor(6., grad_fn=<SumBackward0>)

In [92]:
y.backward()
x.grad

tensor([1., 1., 1., 1.])

In [93]:
y = x*x
y

tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)

In [94]:
#在深度学习中，主要关注的是批量中每个样本单独计算的偏导数之和而不是计算微分矩阵
#对非标量调用backward()需要传入一个“gradient”参数，也就是一个和因变量同形的张量（将张量通过将所有张量的元素加权求和的⽅式转换为标量）
x.grad.zero_()
y = x*x #此时y是张量
y.sum().backward()#等价于y.backward(torch.ones(len(x))),绝大部分时候都只需要对标量求梯度
x.grad  

tensor([0., 2., 4., 6.])

In [95]:
#将某些计算移动到记录的计算图之外（用于固定网络中某些参数）
x.grad.zero_()
y = x*x
u = y.detach() #此时只取y的值，这里的u视为常量
z = u*x
z.sum().backward() #z关于x的梯度是u
x.grad == u

tensor([True, True, True, True])

In [96]:
x.grad.zero_()
y.sum().backward() #y是关于x的函数
x.grad

tensor([0., 2., 4., 6.])

In [97]:
#即使构建函数的计算图需要通过python控制流（例如，条件，循环或者任意函数调用），仍然可以计算得到变量的梯度（pytorch会自动保存计算图）
def f(a):
    b = a*2
    while b.norm() < 1000:
        b = b*2
    if b.sum()>0:
        c = b
    else:
        c = b*100
    return c
a = torch.randn(size=(), requires_grad=True)#a为一个标量
d = f(a)
d.backward()
a,a.grad == d/a

(tensor(0.5946, requires_grad=True), tensor(True))

In [98]:
#可以⽤ with torch.no_grad() 将不想被追踪的操作代码块包裹起来, 
#这种⽅法在评估模型的时候很常⽤，因为在评估模型时，我们并不需要计算可训练参数的梯度。
x.grad.zero_()
x = torch.tensor(1.0, requires_grad=True)
y1 = x ** 2
with torch.no_grad():
     y2 = x ** 3
y3 = y1 + y2
print(x.requires_grad)
print(y1, y1.requires_grad) # True
print(y2, y2.requires_grad) # False
print(y3, y3.requires_grad) # True
y3.backward()
print(x.grad)#关于y2的梯度不会被回传

True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True
tensor(2.)


In [106]:
#如果我们想要修改 tensor 的数值，但是⼜不希望被 autograd 记录（即不会影响反向传播）
#那么我么可以对 tensor.data 进⾏操作
x.grad.zero_()
x = torch.ones(1,requires_grad=True)
print(x.data) #仍然是一个tensor类，
print(x.data.requires_grad)#X已经独立于计算图之外
y = x*x
x.data *= 100 #只改变了值，不会记录在计算图中，所以不会影响梯度传播
y.backward()
print(x) #更改data值会影响tensor值
print(x.grad)

tensor([1.])
False
tensor([100.], requires_grad=True)
tensor([200.])
